# Training,Optimizer,Loss

In [ ]:
#Specify some of the training data as a validation set
#ex:
#https://github.com/Bjarten/early-stopping-pytorch/blob/master/MNIST_Early_Stopping_example.ipynb

### Define EarlyStopping

In [ ]:
#github: Bjarten/early-stopping-pytorch
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

### Define training function

In [4]:
def train(multinet, train_loader, optimizer, losstype, patience, epoch):
    train_losses, validation_losses, avg_train_losses, avg_valid_losses = [[] for i in range(4)]
    earlystop = EarlyStopping(patience=patience,verbose=True) #initialize EarlyStopping object
    for e in range(1,epoch+1):
        multinet.train() #prep model for training
        for batch_idx, (data, target) in enumerate(train_loader):
            optimzer.zero_grad()
            output=multinet(data)
            loss = losstype(output,target)
            loss.backward()
            optimzer.step()
            train_losses.append(loss.item())

        multinet.eval() #prep model for evaluation
        for data, target in valid_loader:
            output=multinet(data)
            loss = losstype(output,target)
            valid_losses.append(loss.item())
    
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(epoch))
        print_msg = (f'[{e:>{epoch_len}}/{epoch:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        earlystop(valid_loss,multinet)
        
        if earlystop.early_stop:
            print("Early stopping")
            break
    
    # load the last checkpoint with the best model
    multinet.load_state_dict(torch.load('checkpoint.pt'))

    return  multinet, avg_train_losses, avg_valid_losses


### Initialize Multinet

In [ ]:
multinet = Multinet()
print(multinet)

### Specify loss function and optimizer

In [ ]:
# specify loss function
losstype = nn.MSELoss() #or softmax that Kara has defined at the end of Architecture

# specify optimizer
optimizer = optim.Adam(multinet.parameters(),lr=1e-3)

### Train the model